In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/title-generation-readers/test_reader.p
/kaggle/input/title-generation-readers/train_reader.p
/kaggle/input/allen-nlp-baseline/__notebook__.ipynb
/kaggle/input/allen-nlp-baseline/res.csv
/kaggle/input/allen-nlp-baseline/submission.csv
/kaggle/input/allen-nlp-baseline/__results__.html
/kaggle/input/allen-nlp-baseline/model.h5
/kaggle/input/allen-nlp-baseline/__output__.json
/kaggle/input/allen-nlp-baseline/custom.css
/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.200d.txt
/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.50d.txt
/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.100d.txt
/kaggle/input/title-generation/test.csv
/kaggle/input/title-generation/train.csv
/kaggle/input/title-generation/vocs.pkl
/kaggle/input/title-generation/sample_submission.csv


In [2]:
!pip install allennlp

     |████████████████████████████████| 30.9MB 36.2MB/s 
     |████████████████████████████████| 3.2MB 24.0MB/s 
     |████████████████████████████████| 92kB 7.9MB/s 
     |████████████████████████████████| 2.1MB 23.5MB/s 
ERROR: en-core-web-sm 2.2.5 has requirement spacy>=2.2.2, but you'll have spacy 2.1.9 which is incompatible.
ERROR: en-core-web-lg 2.2.5 has requirement spacy>=2.2.2, but you'll have spacy 2.1.9 which is incompatible.
  Found existing installation: blis 0.4.1
    Uninstalling blis-0.4.1:
      Successfully uninstalled blis-0.4.1
  Found existing installation: preshed 3.0.2
    Uninstalling preshed-3.0.2:
      Successfully uninstalled preshed-3.0.2
  Found existing installation: plac 1.1.3
    Uninstalling plac-1.1.3:
      Successfully uninstalled plac-1.1.3
  Found existing installation: thinc 7.3.1
    Uninstalling thinc-7.3.1:
      Successfully uninstalled thinc-7.3.1
  Found existing installation: spacy 2.2.3
    Uninstalling spacy-2.2.3:
      Successfully uni

In [3]:
!pip install spacy

In [4]:
from typing import Iterable, Dict, List
import itertools
import torch
import torch.optim as optim
import numpy as np
import random
from sklearn.model_selection import train_test_split
from allennlp.data import Tokenizer, Token
from nltk.tokenize.toktok import ToktokTokenizer
import torch.nn as nn
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper
import spacy
import pickle
from allennlp.data.token_indexers.token_characters_indexer import TokenCharactersIndexer
from allennlp.data import DatasetReader, Instance, Tokenizer, TokenIndexer, Token
from allennlp.data.fields import TextField, ArrayField, MetadataField, SequenceLabelField
from allennlp.data.dataset_readers import DatasetReader
from allennlp.models.encoder_decoders.simple_seq2seq import SimpleSeq2Seq
from allennlp.data.dataset_readers.seq2seq import Seq2SeqDatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.attention import LinearAttention, BilinearAttention, DotProductAttention
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer
from allennlp.common import Params
from allennlp.predictors import SentenceTaggerPredictor
from allennlp.predictors import SimpleSeq2SeqPredictor
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper, StackedSelfAttentionEncoder

In [5]:
SEED = 42

In [6]:
torch.manual_seed(42)
random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [7]:
train_df = pd.read_csv('/kaggle/input/title-generation/train.csv')
train_df.head()

,abstract,title
0,we consider the problem of utility maximizatio...,on optimal investment with processes of long o...
1,in this paper we provide an explicit formula f...,boolean complexes for ferrers graphs
2,"kinesin-5, also known as eg5 in vertebrates is...",relative velocity of sliding of microtubules b...
3,we discuss the transition paths in a coupled b...,bifurcation of transition paths induced by cou...
4,two types of room temperature detectors of ter...,all-electric detectors of the polarization sta...


In [8]:
test_df = pd.read_csv('/kaggle/input/title-generation/test.csv')
test_df.head()

,abstract
0,Most sequence transformation models use recurr...
1,The doc2vec approach was introduced as an exte...
2,LSTM models can vary greatly depending on sequ...
3,A joint learning process of alignment and tran...
4,Current unsupervised image-to-image translatio...


In [9]:
class ArxivTitleDatasetReader(Seq2SeqDatasetReader):
    """
    DatasetReader for Arxiv Papers title generation
    """
    def __init__(self,
                 tokenizer: Tokenizer,
                 token_indexers: Dict[str, TokenIndexer] = None,
                 lazy: bool = False):
        super().__init__(lazy=lazy)
        self.tokenizer = tokenizer
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer("words", lowercase_tokens=True)}

    def text_to_instance(self,
                         abstract_tokens: List[Token],
                         title_tokens: List[Token] = None) -> Instance:
        fields = dict()
        if title_tokens is not None:
            fields["target_tokens"] = TextField(title_tokens, token_indexers=self.token_indexers)
        fields["source_tokens"] = TextField(abstract_tokens, token_indexers=self.token_indexers)
        return Instance(fields)

    def _read(self, file_path: str) -> Iterable[Instance]:
        df = pd.read_csv(file_path)
        for i, (row) in df.iterrows():
            abstract_raw = row["abstract"]
            abstract_tokens = self.tokenizer.tokenize(abstract_raw)
            title = row.get("title")
            title_tokens = None
            if title is not None:
                title_tokens = self.tokenizer.tokenize(title)
            yield self.text_to_instance(abstract_tokens, title_tokens)

In [10]:
path = '/kaggle/input/title-generation/'

In [11]:
class NLTKTokenizer(Tokenizer):

    def __init__(self):
        self.tokenizer = ToktokTokenizer()

    def tokenize(self, text: str) -> List[Token]:
        return [Token(token) for token in self.tokenizer.tokenize(text)]

In [12]:
!python -m spacy download en

     |████████████████████████████████| 11.1MB 1.8MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.1.0-cp36-none-any.whl size=11074435 sha256=9e7d7ad81a290e54a1ddb9c849df55c8ea7b7d80310f97a42894144876303524
  Stored in directory: /tmp/pip-ephem-wheel-cache-hzaqobyh/wheels/39/ea/3b/507f7df78be8631a7a3d7090962194cf55bc1158572c0be77f
Successfully built en-core-web-sm
  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/opt/conda/lib/python3.6/site-packages/en_core_web_sm -->
/opt/conda/lib/python3.6/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [13]:
spacy_en = spacy.load('en')

In [14]:
class spacyTokenizer(Tokenizer):

    def __init__(self):
        self.tokenizer = spacy.load('en')

    def tokenize(self, text: str) -> List[Token]:
        return [Token(tok.text) for tok in self.tokenizer(text) if not tok.text.isspace()]

In [15]:
#token_indexer = {"tokens": SingleIdTokenIndexer(lowercase_tokens=True), "chars": TokenCharactersIndexer("chars", min_padding_length=3)}
tokenizer = spacyTokenizer()

In [16]:
# For the first time execution if we didnt got
# reader = ArxivTitleDatasetReader(tokenizer = tokenizer)

# full_train_dataset = reader.read(path + 'train.csv')
# pickle.dump(full_train_dataset, open('train_reader.p', 'wb'))


# train_dataset, validation_dataset = train_test_split(full_train_dataset, test_size=0.10, random_state=SEED)

# test_dataset = reader.read(path + 'test.csv')
# pickle.dump(test_dataset, open('test_reader.p', 'wb'))

In [17]:
# For the next times execution if we create readers files
reader = ArxivTitleDatasetReader(tokenizer = tokenizer)

full_train_dataset = pickle.load(open('../input/title-generation-readers/train_reader.p', 'rb'))


train_dataset, validation_dataset = train_test_split(full_train_dataset, test_size=0.10, random_state=SEED)

test_dataset = pickle.load(open('../input/title-generation-readers/test_reader.p', 'rb'))

In [18]:
emb_filename = '../input/glove-global-vectors-for-word-representation/glove.6B.200d.txt'

vocab = Vocabulary.from_instances(train_dataset + validation_dataset + test_dataset, 
                                  pretrained_files = {'words': emb_filename},
                                  only_include_pretrained_words=False, min_count={"words": 7})
EMBEDDING_DIM = 200
HIDDEN_DIM = 256
CUDA_DEVICE = 0

100%|██████████| 136000/136000 [00:35<00:00, 3782.74it/s]
400000it [00:02, 168125.41it/s]


In [19]:
instance = train_dataset[0]
instance.index_fields(vocab)
print(instance.as_tensor_dict())
print(vocab)

{'target_tokens': {'tokens': tensor([  134,  5867,     9,  3098, 11149,    32,    17,  4101,   950,    45,
            7,    97,    58])}, 'source_tokens': {'tokens': tensor([   19,    44,   974,     7,    97,    58,     3,     7,  5559,    83,
           14,    30,    26,    67,    10,  7023,   134,     9,  3098, 11149,
          314,  8410,     2, 13329,     3,     2, 13462,     4,    11,    93,
           24,    61,    14,   757,  4101,   262,    28,   113,     6,   757,
          445,    71,    10,   658,  1806,    13, 13462,    10,   927,     2,
          134,  1606,     3,    73, 11149,     4,     2,    83, 23299,    13,
         4016,     6,  1046,   442,     9,     2,   992,    28,   542,    22,
            2,     1,  5582,  6989,    83,     4,    39,   442,    20,  5286,
          115,  1039,  1042,     5, 17298,     6,   263,    16,     2,   393,
          262,     3,     2, 13462,     4,   140,     2,    83,  5735,    24,
         3315,    10,  7023,   134,   314,  2329,    

In [20]:
emb_params = Params({"pretrained_file": emb_filename, 
                     "embedding_dim": 200,
                     "trainable": True,
                     "vocab_namespace": "words"})
token_embedding = Embedding.from_params(vocab, emb_params)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})

400000it [00:08, 46288.71it/s] 


In [21]:
encoder = PytorchSeq2SeqWrapper(nn.LSTM(token_embedding.get_output_dim(), HIDDEN_DIM, batch_first=True, bidirectional=True))


attention = DotProductAttention()

max_decoding_steps = 20   # TODO: make this variable

In [22]:
# model = SimpleSeq2Seq(vocab, word_embeddings,
#                       encoder, max_decoding_steps,
#                       target_embedding_dim=EMBEDDING_DIM,
#                       target_namespace='words',
#                       attention=attention,
#                       beam_size=8,
#                       use_bleu=True,
#                       scheduled_sampling_ratio = 0.5
#                       )

In [23]:
model = torch.load('../input/allen-nlp-baseline/model.h5')

In [24]:
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1

optimizer = optim.Adam(model.parameters(), lr = 0.001)
iterator = BucketIterator(batch_size=32, sorting_keys=[("source_tokens", "num_tokens")])
iterator.index_with(vocab)

trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  num_epochs=30,
                  cuda_device=CUDA_DEVICE,
                  patience = 3,
                  validation_metric="+BLEU")

In [25]:
model.forward_on_instance(train_dataset[0])

{'predictions': array([[ 5867,     9,  3098, 11149,    27,  4101,   950,    45,     7,
            97,    58,    17,    97,    58,    17,    97,    58,     9,
            97,    58],
        [ 5867,     9,  3098, 11149,    32,    17,  4101,   950,    45,
             7,    97,    58,     6,    97,    58,     9,    97,    58,
             9,    97],
        [ 5867,     9,  3098, 11149,    32,    17,  4101,   950,    45,
             7,    97,    58,     6,    97,    58,     9,    97,    58,
             9,  3098],
        [ 5867,     9,  3098, 11149,    32,    17,  4101,   950,    45,
             7,    97,    58,     6,    97,    58,    17,    97,    58,
             9,    97],
        [ 5867,     9,  3098, 11149,    32,    17,  4101,   950,    45,
             7,    97,    58,    17,    97,    58,     9,    97,    58,
             9,    97],
        [ 5867,     9,  3098, 11149,    32,    17,  4101,   950,    45,
             7,    97,    58,    17,    97,    58,     9,    97,    58,
 

In [26]:
trainer.train()

loss: 0.7244 ||: 100%|██████████| 3797/3797 [11:44<00:00,  5.39it/s]
BLEU: 0.1495, loss: 4.8469 ||: 100%|██████████| 422/422 [02:43<00:00,  2.58it/s]
loss: 0.6752 ||: 100%|██████████| 3797/3797 [10:58<00:00,  5.76it/s]
BLEU: 0.1518, loss: 4.8926 ||: 100%|██████████| 422/422 [02:39<00:00,  2.64it/s]
loss: 0.6605 ||: 100%|██████████| 3797/3797 [10:58<00:00,  5.77it/s]
BLEU: 0.1530, loss: 4.8934 ||: 100%|██████████| 422/422 [02:40<00:00,  2.63it/s]
loss: 0.6526 ||: 100%|██████████| 3797/3797 [10:56<00:00,  5.78it/s]
BLEU: 0.1518, loss: 4.9708 ||: 100%|██████████| 422/422 [02:39<00:00,  2.64it/s]
BLEU: 0.1598, loss: 5.0877 ||: 100%|██████████| 422/422 [02:37<00:00,  2.68it/s]
loss: 0.5497 ||:  76%|███████▋  | 2902/3797 [08:25<03:07,  4.77it/s]

In [27]:
torch.save(model, 'model.h5')

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type SimpleSeq2Seq. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BasicTextFieldEmbedder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type PytorchSeq2SeqWrapper. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be c

In [28]:
#model = torch.load('../input/kernel5317bc3674/model.h5')

In [29]:
model.forward_on_instance(train_dataset[1])

{'predictions': array([[   11,   187,    10,   568,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [   54,    10,     2,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [   11,   187,    10,     2,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [   11,   187,    10,   568,     2,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [   11,   187,    10,   275,     3,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [   11,   187,    10,   568,   347,    10, 18257,  1774,  2019,
             9,   780,  3481,    45,     7,   226,     5,     6,    24,
 

In [30]:
#import pandas as pd

#submission_data = pd.read_csv('/kaggle/input/title-generation/test.csv')
#abstracts = submission_data['abstract'].values

In [31]:
def get_csv(model, dataset, df):
    model.eval()
    batch_size = 32
    column = []
    for i in range(len(dataset) // batch_size + 1):
        batch = dataset[i * batch_size: (i + 1) * batch_size]
        predictions = model.forward_on_instances(batch)
        for pred in predictions:
            column.append(' '.join(pred["predicted_tokens"]))
    df["title"] = column
    return df

In [32]:
test_df = pd.read_csv("../input/title-generation/test.csv")
df = get_csv(model, test_dataset, test_df)
df.fillna(' ', inplace = True)
df.to_csv("res.csv", index=False)

In [33]:
df.head()

,abstract,title
0,Most sequence transformation models use recurr...,: deep neural networks : encoder - decoder - f...
1,The doc2vec approach was introduced as an exte...,program with a data - simple and simple contex...
2,LSTM models can vary greatly depending on sequ...,for vertex using symbolic
3,A joint learning process of alignment and tran...,background - based description of alignment an...
4,Current unsupervised image-to-image translatio...,image synthesis with image model image transla...


In [34]:
import string
from nltk.util import ngrams
import numpy as np
import pandas as pd
import pickle


def generate_csv(input_file='res.csv',
                 output_file='submission.csv',
                 voc_file='/kaggle/input/title-generation/vocs.pkl'):
    '''
    Generates file in format required for submitting result to Kaggle
    
    Parameters:
        input_file (str) : path to csv file with your predicted titles.
                           Should have two fields: abstract and title
        output_file (str) : path to output submission file
        voc_file (str) : path to voc.pkl file
    '''
    data = pd.read_csv(input_file)
    with open(voc_file, 'rb') as voc_file:
        vocs = pickle.load(voc_file)

    with open(output_file, 'w') as res_file:
        res_file.write('Id,Predict\n')
        
    output_idx = 0
    for row_idx, row in data.iterrows():
        #print(row)
        trg = row[0]
        trg = trg.translate(str.maketrans('', '', string.punctuation)).lower().split()
        trg.extend(['_'.join(ngram) for ngram in list(ngrams(trg, 2)) + list(ngrams(trg, 3))])
        
        VOCAB_stoi = vocs[row_idx]
        trg_intersection = set(VOCAB_stoi.keys()).intersection(set(trg))
        trg_vec = np.zeros(len(VOCAB_stoi))    

        for word in trg_intersection:
            trg_vec[VOCAB_stoi[word]] = 1

        with open(output_file, 'a') as res_file:
            for is_word in trg_vec:
                res_file.write('{0},{1}\n'.format(output_idx, int(is_word)))
                output_idx += 1


generate_csv()

In [35]:
from IPython.display import FileLink
FileLink('submission.csv')

/kaggle/working/submission.csv

In [36]:
# import pandas as pd
# res = pd.read_csv("../input/res.csv")